In [2]:
import os
import json
from pprint import pprint as pp
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import sklearn
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.manifold import TSNE


%matplotlib inline

In [3]:
df = pd.read_pickle('tag_prop.pkl')
tag_mat = df.as_matrix(columns=None)

In [4]:
df = df.transpose()

In [5]:
tag_names = df.columns.values 

In [6]:

X_reduced = TruncatedSVD(n_components=50, random_state=0).fit_transform(df)
X_embedded = TSNE(n_components=2, perplexity=40, verbose=2).fit_transform(X_reduced)

[t-SNE] Computing pairwise distances...
[t-SNE] Computed conditional probabilities for sample 1000 / 4391
[t-SNE] Computed conditional probabilities for sample 2000 / 4391
[t-SNE] Computed conditional probabilities for sample 3000 / 4391
[t-SNE] Computed conditional probabilities for sample 4000 / 4391
[t-SNE] Computed conditional probabilities for sample 4391 / 4391
[t-SNE] Mean sigma: 0.105719
[t-SNE] Iteration 10: error = 24.3515440, gradient norm = 0.0599939
[t-SNE] Iteration 20: error = 21.7576779, gradient norm = 0.0722222
[t-SNE] Iteration 30: error = 19.6157102, gradient norm = 0.0632508
[t-SNE] Iteration 40: error = 19.3731515, gradient norm = 0.0596437
[t-SNE] Iteration 50: error = 18.7399417, gradient norm = 0.0581769
[t-SNE] Iteration 60: error = 16.2136919, gradient norm = 0.0573181
[t-SNE] Iteration 70: error = 16.1363707, gradient norm = 0.0635388
[t-SNE] Iteration 80: error = 16.3030921, gradient norm = 0.0648152
[t-SNE] Iteration 90: error = 16.7230577, gradient norm =

KeyboardInterrupt: 

In [7]:
cluster_n = 30
k_means = KMeans(n_clusters=cluster_n, init='k-means++', max_iter=100, n_init=1,)
k_means.fit(df)

KMeans(copy_x=True, init='k-means++', max_iter=100, n_clusters=30, n_init=1,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [8]:
k_means.labels_
cluster_bool = list()
for i in range(cluster_n):
    cur_cluster = k_means.labels_ == i
    cluster_bool.append(cur_cluster)

In [11]:
appids = df.index

In [17]:
df_labels = pd.DataFrame({'appid':appids,'cluster':k_means.labels_})
df_labels.to_pickle('df_kmeans.pkl')

In [ ]:
'''red to green color map'''
cdict = {'red':   ((0.0, 0.0, 0.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 1.0, 1.0)),
         'blue':  ((0.0, 0.0, 0.0),
                   (1.0, 0.0, 0.0)),
         'green': ((0.0, 0.0, 1.0),
                   (0.5, 0.0, 0.0),
                   (1.0, 0.0, 0.0))}

cmap = mcolors.LinearSegmentedColormap(
'my_colormap', cdict, 100)
base_val = 100

In [ ]:
qmaps = ['Accent', 'Dark2', 'Paired', 'Pastel1','Pastel2', 'Set1', 'Set2', 'Set3', 'Spectral']
cmap_name = qmaps[8]
cmap=plt.get_cmap(cmap_name)
base_val = 255


In [ ]:
qmaps = ['Accent', 'Dark2', 'Paired', 'Pastel1','Pastel2', 'Set1', 'Set2', 'Set3', 'Spectral']
cmap_name = 'Dark2'
cmap=plt.get_cmap(cmap_name)
base_val = 255

fig,ax = plt.subplots(1,1,figsize=(10, 10))
for i in range(cluster_n):
    cmap_val = int(( (i * 1.0) / (cluster_n - 1)) * base_val)
    ax.plot(X_embedded[cluster_bool[i], 0], X_embedded[cluster_bool[i], 1], marker = '.', ls = '', color = cmap(cmap_val))



In [ ]:
order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]

In [ ]:
'''print out the top labels in each category'''
top_n_labels = 4
order_centroids.shape
for i in range(order_centroids.shape[0]):
    tn = tag_names[order_centroids[i,0:top_n_labels]]
    tn_out = [x.encode('ascii')for x in tn]
    print tn_out

In [ ]:
df